In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999
num_classes=5

In [ ]:
from subprocess import check_output
print(check_output(["ls", "./dataset/regression"]).decode("utf8"))

In [ ]:
train_df = pd.read_csv("./dataset/regression/bank32nh.data", sep=' ', header=None)
train_df=train_df.drop(train_df.columns[-1],axis=1)
print(train_df.shape)

columns=["feat"+str(k) for k in range(train_df.shape[1])]
columns[-1]="label"
train_df.columns=columns

train_df.shape

In [ ]:
train_df.head()

In [ ]:
#train_df['label_ord']=train_df['label']
label=train_df.label.values
sorted_idx=np.argsort(train_df.label.values)
num_samples_per_class=train_df.shape[0]/num_classes
print(num_samples_per_class)

bins=[(k*1e-4+label[sorted_idx[np.round(k*num_samples_per_class-1).astype(np.int)]]) for k in range(1,num_classes+1)]
bins.insert(0,0.0)
bins[-1]=bins[-1]+1
print(bins)

label_ord=label.copy()
for k in range(num_classes):
    print(np.all([label>=bins[k], label<bins[k+1]],0))
    label_ord[np.all([label>=bins[k], label<bins[k+1]],0)]=k
    
print(label_ord)


train_df['label_ord']=label_ord
print(train_df.head())

plt.scatter(label,label_ord)
plt.show()


In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(range(train_df.shape[0]), label[sorted_idx],s=3,c=np.sort(label_ord[sorted_idx]))
plt.xlabel('index', fontsize=12)
plt.ylabel('label', fontsize=12)


In [ ]:
'''ulimit = np.percentile(train_df.label.values, 98)
llimit = np.percentile(train_df.label.values, 2)
train_df['label'].ix[train_df['label']>ulimit] = ulimit
train_df['label'].ix[train_df['label']<llimit] = llimit'''

plt.figure(figsize=(12,8))
sns.distplot(train_df.label.values, bins=50, kde=False)
plt.xlabel('label', fontsize=12)


# Training - Validation Split

In [ ]:
feat=train_df.values[:,:-2]

#Normalize the features

feat_max = np.amax(feat,axis=0)
feat_min = np.amin(feat,axis=0)

feat=(feat-feat_min)/(feat_max-feat_min)
feat=feat*2-1

'''feat_mean = np.mean(feat,axis=0)
feat_std = np.std(feat,axis=0)

feat=(feat-feat_mean)/feat_std
'''
label_ord=train_df.values[:,-1].astype(np.int)

print(np.mean(feat,axis=0))
print(np.min(feat,axis=0))
print(feat.shape)
print(label_ord)

fvec=feat.copy()
#label=np.eye(num_classes)[label_ord]
#print(label)


In [ ]:
test_size = 1000

test_idx=sorted_idx[np.floor(np.linspace(0,len(label_ord)-1,test_size)).astype(np.int)]
train_idx = np.setdiff1d(np.arange(0,len(label_ord)),test_idx)



label_ord_test=label_ord[test_idx]
label_ord_train=label_ord[train_idx]
#label_test=np.eye(num_classes)[label_ord_test]
#label_train=np.eye(num_classes)[label_ord_train]
fvec_test=fvec[test_idx,:]
fvec_train=fvec[train_idx,:]

print(label_ord_test.shape)
print(label_ord_train.shape)
print(fvec_test.shape)
print(fvec_train.shape)

In [ ]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_absolute_error

# Create linear regression object
regr = linear_model.Ridge(alpha = .5)

# Train the model using training sets
regr.fit(fvec_train, label_ord_train)

# Make predictions using the testing set
label_ord_pred = np.round(regr.predict(fvec_test)).astype(np.int)
label_ord_pred[label_ord_pred<0]=0
label_ord_pred[label_ord_pred>=num_classes]=num_classes-1

label_ord_tr_pred = np.round(regr.predict(fvec_train)).astype(np.int)
label_ord_tr_pred[label_ord_tr_pred<0]=0
label_ord_tr_pred[label_ord_tr_pred>=num_classes]=num_classes-1

# The coefficients
print('Coefficients: \n', regr.coef_)

# The mean squared error
print('Training')
print("Mean absolute error: %.2f"
      % mean_absolute_error(label_ord_tr_pred, label_ord_train))

#CCR 
print("Accuracy: %.2f"
      % np.mean(label_ord_tr_pred==label_ord_train))

# The mean squared error
print('Validation')
print("Mean absolute error: %.2f"
      % mean_absolute_error(label_ord_pred, label_ord_test))

#CCR 
print("Accuracy: %.2f"
      % np.mean(label_ord_pred==label_ord_test))

In [ ]:

from sklearn.svm import SVC
clf = SVC()

clf.fit(fvec_train, label_ord_train)
label_ord_pred = clf.predict(fvec_test)
label_ord_tr_pred = clf.predict(fvec_train)
print(np.histogram(label_ord_pred))
print(np.histogram(label_ord_test))


# The coefficients
print('Coefficients: \n', regr.coef_)


# The mean squared error
print('Training')
print("Mean absolute error: %.2f"
      % mean_absolute_error(label_ord_tr_pred, label_ord_train))

#CCR 
print("Accuracy: %.2f"
      % np.mean(label_ord_tr_pred==label_ord_train))

# The mean squared error
print('Validation')
print("Mean absolute error: %.2f"
      % mean_absolute_error(label_ord_pred, label_ord_test))

#CCR 
print("Accuracy: %.2f"
      % np.mean(label_ord_pred==label_ord_test))

# MLP Network

In [ ]:
import tensorflow as tf

label_test=np.eye(num_classes)[label_ord_test]
label_train=np.eye(num_classes)[label_ord_train]

In [ ]:
def mlp(x, hidden_sizes, activation_fn=tf.nn.relu,dropout_rate=.5,std_dev=1.0):
    if not isinstance(hidden_sizes, (list, tuple)):
        raise ValueError("hidden_sizes must be a list or a tuple")
        
    scope_args = {'initializer': tf.random_normal_initializer(stddev=std_dev)}
    
    for k in range(len(hidden_sizes)-1):
        layer_name="weights"+str(k)
        #FC layers
        with tf.variable_scope(layer_name, **scope_args):
            W = tf.get_variable('W', shape=[x.shape[-1], hidden_sizes[k]])
            b = tf.get_variable('b', shape=[hidden_sizes[k]])
            x = activation_fn(tf.matmul(x, W) + b)
            #Dropout before the last layer
            x = tf.nn.dropout(x, keep_prob=dropout_rate)
    #Softmax layer
    with tf.variable_scope('outlayer', **scope_args):
        W = tf.get_variable('W', shape=[x.shape[-1], hidden_sizes[-1]])
        b = tf.get_variable('b', shape=[hidden_sizes[-1]])
        return tf.matmul(x, W) + b

In [ ]:
'''fvec_n=fvec/np.round(np.max(label))
label_n = label/np.round(np.max(label))'''
def test_classification(model_function, learning_rate=0.1,num_iter=30000,num_log=2000):

    with tf.Graph().as_default() as g:
        # where are you going to allocate memory and perform computations
        with tf.device("/gpu:0"):
            
            # define model "input placeholders", i.e. variables that are
            # going to be substituted with input data on train/test time
            x_ = tf.placeholder(tf.float32, [None, fvec.shape[1]])
            y_ = tf.placeholder(tf.float32, [None, num_classes])
            y_logits = model_function(x_)
            

            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_,logits=y_logits))
            '''train_step = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)'''
            train_step = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
           
            y_pred = tf.argmax(y_logits, 1)
            y_true = tf.argmax(y_,1)
            correct_prediction = tf.equal(y_pred, y_true)
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
            mae_error = tf.reduce_mean(tf.cast(tf.abs(y_pred-y_true), tf.float32))

    with g.as_default(), tf.Session() as sess:

        sess.run(tf.global_variables_initializer())
        
        # train
        ids=[i for i in range(128)]
        for iter_i in range(num_iter+1):
            batch_xs = fvec_train[ids,:] 
            batch_ys = label_train[ids]
            ids=[(ids[0]+100+i)%label_train.shape[0] for i in range(100)]
            sess.run(train_step, feed_dict={x_: batch_xs, y_: batch_ys})
            
            # test trained model
            if iter_i % num_log == 0:
                tf_feed_dict = {x_: fvec_train, y_: label_train}
                loss_tr, acc_tr, mae_tr, y_pred_tr,y_true_tr= sess.run(
                    [loss, accuracy, mae_error, y_pred,y_true], feed_dict=tf_feed_dict)
                tf_feed_dict = {x_: fvec_test, y_: label_test}
                loss_val, acc_val, mae_val, y_pred_val,y_true_val= sess.run(
                    [loss, accuracy, mae_error, y_pred,y_true], feed_dict=tf_feed_dict)
                print('iteration %d\t Training: loss: %.5f\t MAE: %.5f\t acc: %.5f\t, Validation: loss: %.5f\t MAE: %.5f\t acc: %.5f\t'%
                      (iter_i, loss_tr, mae_tr, acc_tr, loss_val, mae_val, acc_val))
                '''loss_val= sess.run(loss, feed_dict=tf_feed_dict)
                print('iteration %d\t loss: %.5f\t MAE: %.5f\t acc: %.5f\t'%
                      (iter_i, loss_val, loss_val, loss_val))'''

In [ ]:
test_classification(
    lambda x: mlp(x, [32, 16, 16, 12, 12, 8, 8, 5], activation_fn=tf.nn.relu,std_dev=.1), 
    learning_rate=1e-3,num_iter=100000,num_log=2000)


In [ ]:
test_classification(
    lambda x: mlp(x, [32, 16, 12, 8, 5], activation_fn=tf.nn.relu,std_dev=.1), 
    learning_rate=1e-3,num_iter=500000,num_log=10000)
test_classification(
    lambda x: mlp(x, [32, 16, 12, 8, 5], activation_fn=tf.nn.relu,std_dev=.1), 
    learning_rate=1e-4,num_iter=500000,num_log=10000)
test_classification(
    lambda x: mlp(x, [32, 16, 8, 5], activation_fn=tf.nn.relu,std_dev=.1), 
    learning_rate=1e-3,num_iter=500000,num_log=10000)

In [ ]:
test_classification(
    lambda x: mlp(x, [16, 12, 8, 5], activation_fn=tf.nn.relu,std_dev=.1), 
    learning_rate=1e-3,num_iter=1000000,num_log=10000)


In [ ]:
test_classification(
    lambda x: mlp(x, [16, 12, 8, 5], activation_fn=tf.nn.relu,std_dev=.1), 
    learning_rate=5e-4,num_iter=300000,num_log=10000)